In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers
import numpy as np
import os
import time

import imageio
import glob
import matplotlib.pyplot as plt
from IPython import display
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Flatten, Dense
#from tensorflow_docs.vis import embed
from sklearn.model_selection import train_test_split
#!pip install -q git+https://github.com/tensorflow/docs
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Model
import imageio
from scipy.linalg import sqrtm
from keras.applications.resnet_v2 import ResNet50V2, preprocess_input
from keras.models import Model
from keras.layers import GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from PIL import Image


# Initialize rng
rng = np.random.default_rng(2022)

auc = tf.keras.metrics.AUC()

In [ ]:
batch_size = 32 # This is a tunable hyperparameter
shape = (128, 128) # note we are reducing the size of the image

data_dir = '/content/drive/MyDrive/Eye_Dataset/'

train_ds = tf.keras.utils.image_dataset_from_directory(os.path.join(data_dir, 'train_new'),
                                                       seed=rng.integers(500000),
                                                       image_size=shape,
                                                       label_mode="categorical",
                                                       color_mode='grayscale',
                                                       batch_size=batch_size)
val_ds = tf.keras.utils.image_dataset_from_directory(os.path.join(data_dir, 'validation/validation'),
                                                     seed=rng.integers(500000),
                                                     image_size=shape,
                                                     label_mode="categorical",
                                                     color_mode='grayscale',
                                                     batch_size=batch_size)
test_ds = tf.keras.utils.image_dataset_from_directory(os.path.join(data_dir, 'test_new'),
                                                      seed=rng.integers(500000),
                                                      image_size=shape,
                                                      color_mode='grayscale',
                                                      batch_size=batch_size)

Found 16000 files belonging to 4 classes.
Found 4000 files belonging to 4 classes.
Found 4000 files belonging to 4 classes.


In [ ]:
for x, y in train_ds.take(1):
  print(x.shape)
  print(y.shape)

(32, 128, 128, 1)
(32, 4)


In [ ]:
# for images, labels in train_ds.take(1):
#     print("Images batch shape:", images.shape)
#     print("Labels batch shape:", labels.shape)

In [ ]:
train_dir = '/content/drive/MyDrive/Eye_Dataset/train_new'

In [ ]:
class_names_list = [d for d in os.listdir(train_dir) if os.path.isdir(os.path.join(train_dir, d))]
class_names_list

['DME', 'DRUSEN', 'NORMAL', 'CNV']

In [ ]:
# Number of classes (4 in your case: 'CNV', 'DME', 'DRUSEN', 'NORMAL')
num_classes = len(class_names_list)
num_classes

4

In [ ]:
buffer_size = 16000
batch_size = 32


In [ ]:
# normalize image pixels into range [-1, 1]
def normalize(image, label):
  """
  The normalization function will transform each image pixel value to the range [-1, 1], while the labels remain unchanged.

  """
  image = (image - 127.5) / 127.5
  return image, label

train_ds = train_ds.map(normalize)
val_ds = val_ds.map(normalize)
test_ds = test_ds.map(normalize)


In [ ]:
# Batch and shuffle the data
train_ds = train_ds.shuffle(buffer_size)

#val_ds = val_ds.batch(batch_size)
#test_ds = test_ds.batch(batch_size)


Generator

In [ ]:
def make_generator_model(num_classes):
    model = tf.keras.Sequential()
    model.add(layers.Dense(8*8*1024, use_bias=False, input_shape=(100+num_classes,),
                           kernel_initializer='he_normal'))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    model.add(layers.Reshape((8, 8, 1024)))

    # Upsampling to 16x16
    model.add(layers.Conv2DTranspose(512, (5, 5), strides=(2, 2), padding='same', use_bias=False,
                                     kernel_initializer='he_normal'))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    # Upsampling to 32 x 32
    model.add(layers.Conv2DTranspose(256, (5, 5), strides=(2, 2), padding='same', use_bias=False,
                                     kernel_initializer='he_normal'))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    # Upsampling to 64 X 64
    model.add(layers.Conv2DTranspose(128, (5, 5), strides=(2, 2), padding='same', use_bias=False,
                                     kernel_initializer='he_normal'))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    # Upsampling to 128 X 128
    model.add(layers.Conv2DTranspose(1, (5, 5), strides=(2, 2), padding='same', use_bias=False, activation='tanh',
                                     kernel_initializer='he_normal'))

    return model

Discriminator

In [ ]:
def make_discriminator_model(num_classes):
    model = tf.keras.Sequential()
    model.add(layers.Input(shape=(128, 128, num_classes + 1)))
    model.add(layers.Conv2D(64, (5, 5), strides=(2, 2), padding='same',
                            kernel_initializer='he_normal'))
    model.add(layers.LeakyReLU())
    model.add(layers.Dropout(0.3))

    model.add(layers.Conv2D(128, (5, 5), strides=(2, 2), padding='same',
                            kernel_initializer='he_normal'))
    model.add(layers.LeakyReLU())
    model.add(layers.Dropout(0.3))

    model.add(layers.Conv2D(256, (5, 5), strides=(2, 2), padding='same',
                            kernel_initializer='he_normal'))
    model.add(layers.LeakyReLU())
    model.add(layers.Dropout(0.3))

    model.add(layers.Flatten())
    model.add(layers.Dense(4, kernel_initializer='he_normal'))

    return model


In [ ]:
# Create a batch of one-hot encoded labels
# For simplicity, let's generate random labels. You might want to use specific labels depending on your application.
labels = tf.one_hot(tf.random.uniform([batch_size], minval=0, maxval=num_classes, dtype=tf.int32), depth=num_classes)

# Measure training time
start_time = time.time()

# Prepare labels for concatenation with the images (same as the previous example)
def concatenate_labels(images, labels):
    # Expand the labels to match the image dimensions
    labels_expanded = tf.expand_dims(tf.expand_dims(labels, 1), 2)
    # Tile to match batch, height, and width. No tiling needed for channels.
    labels_expanded = tf.tile(labels_expanded, [1, images.shape[1], images.shape[2], 1])

    # Concatenate the labels with the images
    images_with_labels = tf.concat([images, labels_expanded], axis=-1)
    return images_with_labels

end_time = time.time()
total_time = (end_time - start_time)/60
print(f"Training time: {total_time} mins")

Training time: 2.5828679402669272e-06 mins


In [ ]:
# Parameters (modify these as per your requirements)
latent_dim = 100  # Size of the latent noise vector
num_classes = 4   # Number of classes for conditional GAN
batch_size = 32   # Batch size for training
num_epochs = 10   # Number of epochs for training
learning_rate = 0.0002  # Learning rate for optimizers

# Initialize the generator and discriminator
generator = make_generator_model(num_classes)
discriminator = make_discriminator_model(num_classes)

# Initialize optimizer with gradient clipping
g_optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate, clipvalue=1.0)
d_optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate, clipvalue=1.0)

# Loss function
loss_fn = tf.keras.losses.BinaryCrossentropy(from_logits=True)

# Measure training time
start_time = time.time()
print("start ", start_time)
# Training loop
for epoch in range(num_epochs):
    print(f"Starting Epoch {epoch}")
    for batch, (real_images, labels) in enumerate(train_ds):
        print(f"Processing Batch {batch}")

        # Generate random noise
        noise = tf.random.normal([batch_size, latent_dim])

        # Prepare labels for generator input
        labels_for_generator = tf.reshape(labels, [batch_size, num_classes])
        combined_input = tf.concat([noise, labels_for_generator], axis=1)

        # Generate fake images
        generated_images = generator(combined_input, training=True)

        # Expand and tile labels to match image dimensions
        labels_expanded = tf.reshape(labels, [batch_size, 1, 1, num_classes])
        labels_tiled = tf.tile(labels_expanded, [1, 128, 128, 1])

        # Concatenate labels with real and generated images
        real_images_with_labels = tf.concat([real_images, labels_tiled], axis=-1)
        generated_images_with_labels = tf.concat([generated_images, labels_tiled], axis=-1)

        # Train Discriminator
        with tf.GradientTape() as disc_tape:
            real_output = discriminator(real_images_with_labels, training=True)
            fake_output = discriminator(generated_images_with_labels, training=True)

            d_loss_real = loss_fn(tf.ones_like(real_output), real_output)
            d_loss_fake = loss_fn(tf.zeros_like(fake_output), fake_output)
            d_loss = d_loss_real + d_loss_fake

        gradients_of_discriminator = disc_tape.gradient(d_loss, discriminator.trainable_variables)
        d_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))

        # Train Generator
        with tf.GradientTape() as gen_tape:
            generated_images_with_labels = tf.concat([generated_images, labels_tiled], axis=-1)
            fake_output = discriminator(generated_images_with_labels, training=True)
            g_loss = loss_fn(tf.ones_like(fake_output), fake_output)

        gradients_of_generator = gen_tape.gradient(g_loss, generator.trainable_variables)
        for grad, var in zip(gradients_of_generator, generator.trainable_variables):
            if grad is None:
                print(f"None gradient in generator for variable: {var.name}")
        print("Gradients of Generator:", gradients_of_generator)  # Debugging statement

        # Check if any gradient is None
        if any(grad is None for grad in gradients_of_generator):
            raise ValueError("Encountered None gradient. Check model architecture and loss function.")

        g_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))

        # Print losses to monitor the training process
        if batch % 10 == 0:  # Adjust the interval as needed
            print(f"Epoch {epoch}, Batch {batch}, Gen Loss: {g_loss.numpy()}, Disc Loss: {d_loss.numpy()}")

    # Save models at regular intervals (optional)
    if epoch % 1 == 0:  # Adjust the interval as needed
        generator.save(f'generator_epoch_{epoch}.h5')
        discriminator.save(f'discriminator_epoch_{epoch}.h5')
        print(f"End of Epoch {epoch}, Generator Loss: {g_loss.numpy()}, Discriminator Loss: {d_loss.numpy()}")

end_time = time.time()
total_time = (end_time - start_time)/60
print(f"Training time: {total_time} mins")


start  1702012511.7888947
Starting Epoch 0
Processing Batch 0
None gradient in generator for variable: dense_24/kernel:0
None gradient in generator for variable: batch_normalization_48/gamma:0
None gradient in generator for variable: batch_normalization_48/beta:0
None gradient in generator for variable: conv2d_transpose_48/kernel:0
None gradient in generator for variable: batch_normalization_49/gamma:0
None gradient in generator for variable: batch_normalization_49/beta:0
None gradient in generator for variable: conv2d_transpose_49/kernel:0
None gradient in generator for variable: batch_normalization_50/gamma:0
None gradient in generator for variable: batch_normalization_50/beta:0
None gradient in generator for variable: conv2d_transpose_50/kernel:0
None gradient in generator for variable: batch_normalization_51/gamma:0
None gradient in generator for variable: batch_normalization_51/beta:0
None gradient in generator for variable: conv2d_transpose_51/kernel:0
Gradients of Generator: [No

ValueError: ignored

In [ ]:
# # Define real_images and real_labels
# real_images, labels = train_ds
# real_images_with_labels = concatenate_labels(real_images, labels)
# fake_images_with_labels = concatenate_labels(generated_images, labels)

# # Then use these concatenated images as input discriminator
# real_output = discriminator(real_images_with_labels, training=True)
# fake_output = discriminator(fake_images_with_labels, training=True)

In [ ]:
# Training hyerparameters
EPOCHS = 50
noise_dim = 100
num_examples_to_generate = 16
latent_dim = 100
# You will reuse this seed overtime (so it's easier)
# to visualize progress in the animated GIF)
seed = tf.random.normal([num_examples_to_generate, noise_dim])

In [ ]:
# # Visualize one of the generated images (for example, the first image in the batch)
# plt.imshow(generated_images[0, :, :, 0], cmap='gray')
# plt.axis('off')
# plt.show()

In [ ]:
class cGAN(tf.keras.Model):
    def __init__(self, discriminator, generator, latent_dim, batch_size):
        super(cGAN, self).__init__()
        self.discriminator = discriminator
        self.generator = generator
        self.latent_dim = latent_dim
        self.batch_size = batch_size
        self.num_classes = 4  # Adjusted for 4-class conditional GAN


    def compile(self, d_optimizer, g_optimizer, loss_fn):
        super(cGAN, self).compile()
        self.d_optimizer = d_optimizer
        self.g_optimizer = g_optimizer
        self.loss_fn = loss_fn
        self.d_loss_metric = tf.keras.metrics.Mean(name="d_loss")
        self.g_loss_metric = tf.keras.metrics.Mean(name="g_loss")

    @property
    def metrics(self):
        return [self.d_loss_metric, self.g_loss_metric]

    def generator_loss(self, fake_output):
        return self.loss_fn(tf.ones_like(fake_output), fake_output)

    def discriminator_loss(self, real_output, fake_output):
        real_loss = self.loss_fn(tf.ones_like(real_output), real_output)
        fake_loss = self.loss_fn(tf.zeros_like(fake_output), fake_output)
        total_loss = real_loss + fake_loss
        return total_loss
    def train_step(self, data):
        real_images, labels = data

        # Reshape and expand labels to match image dimensions
        labels_reshaped = tf.reshape(labels, [self.batch_size, 1, 1, self.num_classes])
        labels_expanded = tf.tile(labels_reshaped, [1, 128, 128, 1])

        # Generate random noise
        noise = tf.random.normal([self.batch_size, self.latent_dim])
        combined_input = tf.concat([noise, tf.reshape(labels, [self.batch_size, self.num_classes])], axis=1)

        # Generate fake images
        generated_images = self.generator(combined_input, training=True)

        # Concatenate labels with real and fake images
        real_images_with_labels = tf.concat([real_images, labels_expanded], axis=-1)
        fake_images_with_labels = tf.concat([generated_images, labels_expanded], axis=-1)

        with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
            # Discriminator predictions
            real_output = self.discriminator(real_images_with_labels, training=True)
            fake_output = self.discriminator(fake_images_with_labels, training=True)

            # Loss calculations
            gen_loss = self.generator_loss(fake_output)
            disc_loss = self.discriminator_loss(real_output, fake_output)

        # Gradient calculations with gradient checking
        gen_loss += 1e-8  # Gradient checking
        gradients_of_generator = gen_tape.gradient(gen_loss, self.generator.trainable_variables)
        gradients_of_discriminator = disc_tape.gradient(disc_loss, self.discriminator.trainable_variables)

        if any(g is None for g in gradients_of_generator):
            print("One or more gradients are None for the generator.")

        # Apply gradients
        self.g_optimizer.apply_gradients(zip(gradients_of_generator, self.generator.trainable_variables))
        self.d_optimizer.apply_gradients(zip(gradients_of_discriminator, self.discriminator.trainable_variables))

        # Update metrics
        self.d_loss_metric.update_state(disc_loss)
        self.g_loss_metric.update_state(gen_loss)

        return {
            "d_loss": self.d_loss_metric.result(),
            "g_loss": self.g_loss_metric.result(),
        }


In [ ]:
class cGANMonitor(tf.keras.callbacks.Callback):

    def __init__(self, num_img=16, latent_dim=latent_dim, num_eval_img=1000):
        super().__init__()
        self.num_img = num_img
        self.latent_dim = latent_dim
        self.real_vs_fake_accuracy = []
        self.num_eval_img = num_eval_img  # Number of images to evaluate for real vs. fake classification
        self.d_losses = []
        self.g_losses = []

    def on_epoch_end(self, epoch, logs=None):
        self.d_losses.append(logs['d_loss'])
        self.g_losses.append(logs['g_loss'])

        random_latent_vectors = tf.random.normal(shape=(self.num_img, self.latent_dim))
        predictions = self.model.generator(random_latent_vectors, training=False)

        fig = plt.figure(figsize=(4, 4))
        for i in range(predictions.shape[0]):
            plt.subplot(4, 4, i+1)
            plt.imshow((predictions[i] + 1) / 2)  # Assumes 'tanh' activation in the last layer
            plt.axis('off')
        plt.savefig('image_at_epoch_{:04d}.png'.format(epoch))
        plt.close(fig)  # Close the figure after saving to free up memory



        # Generate a batch of fake images
        random_latent_vectors = tf.random.normal(shape=(self.num_eval_img, self.latent_dim))
        fake_images = self.model.generator(random_latent_vectors, training=False)

        # Get a batch of real images and add a batch dimension if necessary
        real_images = next(iter(train_dataset.take(1)))[0]
        if len(real_images.shape) == 3:
            real_images = tf.expand_dims(real_images, 0)  # Add batch dimension if it's not there

        # Ensure the real_images has the batch size you expect (e.g., self.num_eval_img)
        real_images = real_images[:self.num_eval_img]

        # Classify real and fake images
        real_predictions = self.model.discriminator(real_images, training=False)
        fake_predictions = self.model.discriminator(fake_images, training=False)

        # Calculate the accuracy
        real_accuracy = np.mean(real_predictions > 0.5)
        fake_accuracy = np.mean(fake_predictions <= 0.5)
        total_accuracy = (real_accuracy + fake_accuracy) / 2

        # Log the accuracy
        self.real_vs_fake_accuracy.append(total_accuracy)


In [ ]:
# print("Generator Summary:")
# generator.summary()

# print("Discriminator Summary:")
# discriminator.summary()

In [ ]:
# # Example data (fake_output and real_output)
# # Replace with actual discriminator outputs for a batch of data
# fake_output = discriminator(fake_images_with_labels, training=True)
# real_output = discriminator(real_images_with_labels, training=True)
# # Calculate generator loss
# gen_loss = generator_loss(fake_output)
# print("Generator Loss:", gen_loss.numpy())

# # Calculate discriminator loss
# disc_loss = discriminator_loss(real_output, fake_output)
# print("Discriminator Loss:", disc_loss.numpy())


In [ ]:
# Define and compile the generator and discriminator models
generator = make_generator_model(num_classes)
discriminator = make_discriminator_model(num_classes)

# Check if the models have trainable variables
if not generator.trainable_variables:
    raise ValueError("Generator has no trainable variables. Make sure it's properly built and compiled.")

if not discriminator.trainable_variables:
    raise ValueError("Discriminator has no trainable variables. Make sure it's properly built and compiled.")


#creates an instance of GAN model
gan = cGAN(discriminator=discriminator, generator=generator, latent_dim=latent_dim, batch_size=batch_size)
gan.compile(
    d_optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),
    g_optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),
    loss_fn=tf.keras.losses.BinaryCrossentropy(),
)

# Measure training time
start_time = time.time()

# After training GAN model with the GANMonitor callback:
monitor = cGANMonitor(num_img=16, latent_dim=latent_dim)
gan.fit(train_ds,
        epochs=EPOCHS,
        callbacks=[monitor])

# Plot the real vs. fake classification accuracy
plt.figure(figsize=(10, 5))
plt.plot(range(1, len(monitor.real_vs_fake_accuracy) + 1), monitor.real_vs_fake_accuracy, label='Real vs. Fake Accuracy', color='purple')
plt.title('Discriminator Real vs. Fake Classification Accuracy Over Epochs')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True)
plt.show()

end_time = time.time()
total_time = (end_time - start_time)/60
print(f"Training time: {total_time} mins")



After reshape: (None, 8, 8, 1024)
After first upsample: (None, 16, 16, 512)
After second upsample: (None, 32, 32, 256)
After third upsample: (None, 64, 64, 128)
After fourth upsample: (None, 128, 128, 1)
Epoch 1/50
One or more gradients are None for the generator.


ValueError: ignored

In [ ]:
gan.save('gan_model')  # 'gan_model' save the model

In [ ]:
for images, labels in train_ds.take(1):
    print("Images batch shape:", images.shape)
    print("Labels batch shape:", labels.shape)
